# Developing Hallucination Guardrails

In this notebook we'll walk through the process of developing a guardrail that checks model outputs and against hallucinations. 

This notebook will focus on:
1. Building out a strong eval set
2. Identifying specific criteria to measure hallucinations
3. Improving accuracy with few-shot prompting

In [1]:
import json
from openai import OpenAI
import pandas as pd
from IPython.display import display, HTML
from concurrent.futures import ThreadPoolExecutor
import os

client = OpenAI()

# 1. Building out an eval set

Imagine we are a customer support team that is building out an automated support agent. We will be feeding the assistant information from our knowledge base about a specific set of policies for how to handle tickets such as returns, refunds, feedback, and expect the model to follow the policy when interacting with customers.

The first thing we will do is use GPT-4 to build out a set of policies that we will want to follow.

If you want to do deep dive into generating synthetic data, you can review our Synthetic Data Generation Cookbook [here](https://cookbook.openai.com/examples/sdg1)

In [2]:
system_input_prompt = """"
You are a helpful assistant that can generate policies for a support agent at a fictional company to follow. 
You will be provided with a topic (ie. returns, refunds, feedback) and you are to generate a sample policy for how to handle the it.

When constructing the policy, it should contain step-by-step instructions for how to handle the customer inquiry. 
It should include decision logic for what to do if a customer falls under a certain category, and provide requirements for taking specific actions.
"""

user_policy_example_1 = """"
RETURN POLICY
"""

assistant_policy_example_1 = """
RETURN POLICY

1. ** Ask the customer why they want the order replaced **
    - Categorize their issue into one of the following buckets:
        - damaged: They received the product in a damaged state
        - satisfaction: The customer is not satisfied with the item and does not like the product.
        - unnecessary: They no longer need the item
2a. **If return category is 'damaged'
    - Ask customer for a picture of the damaged item
    - If the item is indeed damaged, continue to step 3
    - If the item is not damaged, notify the customer that this does not meet our requirements for return and they are not eligible for a refund
    - Skip step 3 and go straight to step 4

2b. **If return category is either 'satisfaction' or 'unnecessary'**
    - Ask the customer if they can provide feedback on the quality of the item
    - If the order was made within 30 days, notify them that they are eligible for a full refund
    - If the order was made within 31-60 days, notify them that they are eligible for a partial refund of 50%
    - If the order was made greater than 60 days ago, notify them that they are not eligble for a refund

3. **If the customer is eligible for a return or refund**
    - Ask the customer to confirm that they would like a return or refund
    - Once they confirm, process their request

4 **Provide additional support before closing out ticket**
    - Ask the customer if there is anything else you can do to help them today.

"""

user_policy_input = """
{{POLICY}}
"""

In [3]:
policy_instructions = []
def generate_policies():
    policies = ['PRODUCT FEEDBACK POLICY', 'SHIPPING POLICY', 'WARRANTY POLICY', 'ACCOUNT DELETION', 'COMPLAINT RESOLUTION']
    for policy in policies:
        input_message = user_policy_input.replace("{{POLICY}}", policy)
        response = client.chat.completions.create(
            messages= [
                {"role": "system", "content": system_input_prompt},
                {"role": "user", "content": user_policy_example_1},
                {"role": "assistant", "content": assistant_policy_example_1},
                {"role": "user", "content": input_message},
            ],
            model="gpt-4o"
        )
        policy_instructions.append(response.choices[0].message.content)
        print(response.choices[0].message.content)

generate_policies()

**PRODUCT FEEDBACK POLICY**

1. **Acknowledge the Feedback Request**
   - Thank the customer for their interest in providing feedback.
   - Ensure them that their input is valuable and will help improve the product and services.

2. **Categorize the Feedback**
   - Determine the nature of the feedback:
     - Positive feedback
     - Negative feedback
     - Suggestions for improvement
     - Bug or defect report
   - Based on categorization, follow the corresponding steps:

3. **Handle Positive Feedback**
   - Express appreciation for the positive feedback.
   - If applicable, ask for permission to use their feedback in marketing materials or on the website (with appropriate confidentiality considerations).

4. **Handle Negative Feedback**
   - Apologize for any inconvenience or dissatisfaction caused.
   - Collect detailed information about the issue, such as:
     - Specific problems encountered
     - Context of usage
     - Expectations not met
   - Reassure the customer that thei

Next we'll take these policies and generate sample customer interactions that do or do not follow the instructions.

In [4]:
system_input_prompt = """"
You are a helpful assistant that can generate fictional interactions between a support assistant and a customer user. 
You will be given a set of policy instructions that the support agent is instructed to follow.

Based on the instructions, you must generate a relevant single-turn or multi-turn interaction between the assistant and the user. It should average between 1-3 turns total.
For a given set of instructions, generate an example conversation that where the assistant either does or does not follow the instructions properly. In the assistant's responses, have it give a combination of single sentence and multi-sentence responses.

The output must be in a json format with the following three parameters:
 - accurate: 
    - This should be a boolean true or false value that matches whether or not the final assistant message accurately follows the policy instructions
 - kb_article:
    - This should be the entire policy instruction that is passed in from the user
 - chat_history: 
    - This should contain the entire conversation history except for the final assistant message. 
    - This should be in a format of an array of jsons where each json contains two parameters: role, and content. 
    - Role should be set to either 'user' to represent the customer, or 'assistant' to represent the customer support assistant. 
    - Content should contain the message from the appropriate role.
    - The final message in the chat history should always come from the user. The assistant response in the following parameter will be a response to this use message.
 - assistant_response: 
    - This should contain the final response from the assistant. This is what we will evaluate to determine whether or not it is accurately following the policy.
"""

user_example_1 = """"
Here are the policy instructions:
RETURN POLICY

1. ** Ask the customer why they want the order replaced **
    - Categorize their issue into one of the following buckets:
        - damaged: They received the product in a damaged state
        - satisfaction: The customer is not satisfied with the item and does not like the product.
        - unnecessary: They no longer need the item
2a. **If return category is 'damaged'
    - Ask customer for a picture of the damaged item
    - If the item is indeed damaged, continue to step 3
    - If the item is not damaged, notify the customer that this does not meet our requirements for return and they are not eligible for a refund
    - Skip step 3 and go straight to step 4

2b. **If return category is either 'satisfaction' or 'unnecessary'**
    - Ask the customer if they can provide feedback on the quality of the item
    - If the order was made within 30 days, notify them that they are eligible for a full refund
    - If the order was made within 31-60 days, notify them that they are eligible for a partial refund of 50%
    - If the order was made greater than 60 days ago, notify them that they are not eligble for a refund

3. **If the customer is eligible for a return or refund**
    - Ask the customer to confirm that they would like a return or refund
    - Once they confirm, process their request

4 **Provide additional support before closing out ticket**
    - Ask the customer if there is anything else you can do to help them today.
"""

assistant_example_1 = """
{
    "accurate": "true",
    "kb_article": "1. ** Ask the customer why they want the order replaced ** - Categorize their issue into one of the following buckets: - damaged: They received the product in a damaged state - satisfaction: The customer is not satisfied with the item and does not like the product. - unnecessary: They no longer need the item 2a. **If return category is 'damaged' - Ask customer for a picture of the damaged item - If the item is indeed damaged, continue to step 3 - If the item is not damaged, notify the customer that this does not meet our requirements for return and they are not eligible for a refund - Skip step 3 and go straight to step 4 2b. **If return category is either 'satisfaction' or 'unnecessary'** - Ask the customer if they can provide feedback on the quality of the item - If the order was made within 30 days, notify them that they are eligible for a full refund - If the order was made within 31-60 days, notify them that they are eligible for a partial refund of 50% - If the order was made greater than 60 days ago, notify them that they are not eligble for a refund 3. **If the customer is eligible for a return or refund** - Ask the customer to confirm that they would like a return or refund - Once they confirm, process their request 4 **Provide additional support before closing out ticket** - Ask the customer if there is anything else you can do to help them today.",
    "chat_history": [
        {
            "role": "user",
            "content: "I would like to return this shirt"
        },
        {
            "role": "assistant",
            "content": "Hi there, I'm happy to help with processing this return. Can you please provide an explanation for why you'd like to return this shirt?"
        },
        {
            "role": "user",
            "content: "Yes, I am not satisfied with the design"
        }
    ],
    "assistant_response": {
        "role": "assistant",
        "content": "I see. Because the shirt was ordered in the last 30 days, we can provide you with a full refund. Would you like me to process the refund?"
    }
}
"""

user_example_2 = """"
Here are the policy instructions:
RETURN POLICY

1. ** Ask the customer why they want the order replaced **
    - Categorize their issue into one of the following buckets:
        - damaged: They received the product in a damaged state
        - satisfaction: The customer is not satisfied with the item and does not like the product.
        - unnecessary: They no longer need the item
2a. **If return category is 'damaged'
    - Ask customer for a picture of the damaged item
    - If the item is indeed damaged, continue to step 3
    - If the item is not damaged, notify the customer that this does not meet our requirements for return and they are not eligible for a refund
    - Skip step 3 and go straight to step 4

2b. **If return category is either 'satisfaction' or 'unnecessary'**
    - Ask the customer if they can provide feedback on the quality of the item
    - If the order was made within 30 days, notify them that they are eligible for a full refund
    - If the order was made within 31-60 days, notify them that they are eligible for a partial refund of 50%
    - If the order was made greater than 60 days ago, notify them that they are not eligble for a refund

3. **If the customer is eligible for a return or refund**
    - Ask the customer to confirm that they would like a return or refund
    - Once they confirm, process their request

4 **Provide additional support before closing out ticket**
    - Ask the customer if there is anything else you can do to help them today.
"""

assistant_example_2 = """
{
    "accurate": "false",
    "kb_article": "1. ** Ask the customer why they want the order replaced ** - Categorize their issue into one of the following buckets: - damaged: They received the product in a damaged state - satisfaction: The customer is not satisfied with the item and does not like the product. - unnecessary: They no longer need the item 2a. **If return category is 'damaged' - Ask customer for a picture of the damaged item - If the item is indeed damaged, continue to step 3 - If the item is not damaged, notify the customer that this does not meet our requirements for return and they are not eligible for a refund - Skip step 3 and go straight to step 4 2b. **If return category is either 'satisfaction' or 'unnecessary'** - Ask the customer if they can provide feedback on the quality of the item - If the order was made within 30 days, notify them that they are eligible for a full refund - If the order was made within 31-60 days, notify them that they are eligible for a partial refund of 50% - If the order was made greater than 60 days ago, notify them that they are not eligble for a refund 3. **If the customer is eligible for a return or refund** - Ask the customer to confirm that they would like a return or refund - Once they confirm, process their request 4 **Provide additional support before closing out ticket** - Ask the customer if there is anything else you can do to help them today.",
    "chat_history": [
        {
            "role": "user",
            "content: "I would like to return this shirt"
        },
        {
            "role": "assistant",
            "content": "Hi there, I'm happy to help with processing this return. Can you please provide an explanation for why you'd like to return this shirt?"
        },
        {
            "role": "user",
            "content: "Yes, I am not satisfied with the design"
        }
    ],
    "assistant_response": {
        "role": "assistant",
        "content": "I see. Because the shirt was ordered in the last 60 days, we cannot process a refund."    
    }
}
"""

Now lets iterate through the policies and generate some examples.

In [5]:
customer_interactions = []

def fetch_response(policy):
    messages = [
        { "role": "system", "content": system_input_prompt},
        { "role": "user", "content": user_example_1},
        { "role": "assistant", "content": assistant_example_1},
        { "role": "user", "content": user_example_2},
        { "role": "assistant", "content": assistant_example_2},
        { "role": "user", "content": policy}
    ]

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=messages,
        temperature=0.7,
        n=10
    )
    return response.choices

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(fetch_response, policy) for policy in policy_instructions]
    for future in futures:
        choices = future.result()
        customer_interactions.extend([choice.message.content for choice in choices])


In [6]:
print(customer_interactions)

In [7]:
# Function to clean and fix JSON strings
def clean_json_string(json_str):
    try:
        # Attempt to load the JSON string to identify issues
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        # Fix common issues like extra commas or unescaped characters
        json_str = json_str.replace('\n', ' ').replace('\r', '')
        return json.loads(json_str)
    

# Clean and decode the JSON strings
data = [clean_json_string(entry) for entry in customer_interactions]
df = pd.DataFrame(data)

In [8]:
# Function to set up display options for pandas
def setup_pandas_display():
    # Increase display limits
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)

# Function to make DataFrame scrollable in the notebook output
def make_scrollable(df):
    style = (
        '<style>'
        'div.output_scroll {'
        'resize: both;'
        'overflow: auto;'
        '}'
        '</style>'
    )
    html = f"{style}{df.to_html()}"
    display(HTML(html))

# Main function to display DataFrame
def display_dataframe(df):
    setup_pandas_display()    # Enable scrollable view
    make_scrollable(df)

display_dataframe(df)


,accurate,kb_article,chat_history,assistant_response
0,True,"PRODUCT FEEDBACK POLICY 1. **Acknowledge the Feedback Request** - Thank the customer for their interest in providing feedback. - Ensure them that their input is valuable and will help improve the product and services. 2. **Categorize the Feedback** - Determine the nature of the feedback: - Positive feedback - Negative feedback - Suggestions for improvement - Bug or defect report - Based on categorization, follow the corresponding steps: 3. **Handle Positive Feedback** - Express appreciation for the positive feedback. - If applicable, ask for permission to use their feedback in marketing materials or on the website (with appropriate confidentiality considerations). 4. **Handle Negative Feedback** - Apologize for any inconvenience or dissatisfaction caused. - Collect detailed information about the issue, such as: - Specific problems encountered - Context of usage - Expectations not met - Reassure the customer that their feedback will be forwarded to the product development team. 5. **Handle Suggestions for Improvement** - Thank the customer for their constructive suggestions. - Gather detailed information about the suggested features or improvements. - Inform the customer that their suggestion will be reviewed by the relevant teams for consideration in future updates. 6. **Handle Bug or Defect Reports** - Apologize for the inconvenience caused by the bug or defect. - Gather detailed information about the issue, including: - Steps to reproduce the bug - Device or environment where the issue occurred - Any error messages or screenshots - Assure the customer that the information will be passed on to the technical team for resolution. - Provide an estimated timeline, if possible, for when a fix or update might be released. 7. **Escalate if Necessary** - If the feedback highlights a serious issue or requires immediate attention, escalate it to the relevant department or senior staff. - Inform the customer about the escalation and what actions will be taken next. 8. **Provide Follow-Up Information** - If requested or if the feedback is particularly important, follow up with the customer once their feedback has been reviewed or acted upon. - Provide updates on any changes implemented or fixes made as a result of their feedback. 9. **Log the Feedback** - Record the feedback in the appropriate system or database for future reference. - Categorize and annotate the feedback for easy access and tracking. 10. **Conclude the Interaction** - Ask the customer if there is anything else you can assist with. - Thank the customer again for their valuable feedback and support.","[{'role': 'user', 'content': 'I've been using your software and it's been crashing a lot lately. It's frustrating!'}, {'role': 'assistant', 'content': 'Thank you for reaching out and providing your feedback. I'm really sorry to hear about the issues you've been facing with our software. To help our technical team resolve this, could you please provide more details about the problems you encountered, such as when it typically crashes and what you were doing at the time?'}]","{'role': 'assistant', 'content': 'Also, if you could let us know the device or operating system you're using, and any error messages you received, that would be extremely helpful. This information will be forwarded to our technical team right away to diagnose and address the issue. I appreciate your patience and assistance in resolving this matter. Can I do anything else for you today?'}"
1,True,"**PRODUCT FEEDBACK POLICY** 1. **Acknowledge the Feedback Request** - Thank the customer for their interest in providing feedback. - Ensure them that their input is valuable and will help improve the product and services. 2. **Categorize the Feedback** - Determine the nature of the feedback: - Positive feedback - Negative feedback - Suggestions for improvement - Bug or defect report - Based on categorization, follow the corresponding steps: 3. **Handle 

# 2. Constructing our hallucination guardrail

When building out our hallucination guardrail, here are some guiding principles:

1. Provide very descriptive metrics to evaluate whether or not a response is accurate
- It is important to break down this idea of "truth" in easily identifiable metrics that we can measure
- Metrics like truthfulness and relevance are difficult to measure. Giving concrete ways to score the statement can result in a more accurate guardrail
2. Ensure consistency across key terminology
- It is important to keep relevant terms such as KB articles, assistants, and users consistent across the prompt
- If we begin to use phrases such as assistant vs agent, the model could get confused
3. Start with the most advanced model
- GPT-4-Turbo may be the most expensive model, but it important to start with the most advanced so we can ensure a high degree of accuracy
- Once we have thoroughly tested out the guardrail and are confident in its performance, we can look to reducing cost by tuning it down to a 3.5 model
4. Evaluate each sentence independently and the entire response as a whole
- If the agent returns a long response, it can be useful to break down the response to individual sentences and evaluate them independently
- In addition to that, evaluating the whole intent of the message as a whole can ensure that you don't lose important context

With all of this in mind, let's build out a guardrail system and measure its performance.

In [9]:
guardrail_system_message = """You are a highly specialized assistant tasked with reviewing chatbot responses to identify and flag any inaccuracies or hallucinations. For each user message, you must thoroughly analyze the response by considering:
    1. Knowledge Accuracy: Does the message accurately reflect information found in the knowledge base? Assess not only direct mentions but also contextually inferred knowledge.
    2. Relevance: Does the message directly address the user's question or statement? Check if the response logically follows the user’s last message, maintaining coherence in the conversation thread.
    3. Policy Compliance: Does the message adhere to company policies? Evaluate for subtleties such as misinformation, overpromises, or logical inconsistencies. Ensure the response is polite, non-discriminatory, and practical.

To perform your task you will be given the following:
    1. Knowledge Base Articles - These are your source of truth for verifying the content of assistant messages.
    2. Chat Transcript - Provides context for the conversation between the user and the assistant.
    3. Assistant Message - The message from the assistant that needs review.

For each sentence in the assistant's most recent response, assign a score based on the following criteria:
    1. Factual Accuracy:
        - Score 1 if the sentence is factually correct and corroborated by the knowledge base.
        - Score 0 if the sentence contains factual errors or unsubstantiated claims.
    2. Relevance:
        - Score 1 if the sentence directly and specifically addresses the user's question or statement without digression.
        - Score 0 if the sentence is tangential or does not build logically on the conversation thread.
    3. Policy Compliance:
        - Score 1 if the response complies with all company policies including accuracy, ethical guidelines, and user engagement standards.
        - Score 0 if it violates any aspect of the policies, such as misinformation or inappropriate content.
    4. Contextual Coherence:
        - Score 1 if the sentence maintains or enhances the coherence of the conversation, connecting logically with preceding messages.
        - Score 0 if it disrupts the flow or context of the conversation.

Include in your response an array of JSON objects for each evaluated sentence. Each JSON object should contain:
    - `sentence`: Text of the evaluated sentence.
    - `factualAccuracy`: Score for factual correctness (0 or 1).
    - `factualReference`: If scored 1, cite the exact line(s) from the knowledge base. If scored 0, provide a rationale.
    - `relevance`: Score for relevance to the user’s question (0 or 1).
    - `policyCompliance`: Score for adherence to company policies (0 or 1).
    - `contextualCoherence`: Score for maintaining conversation coherence (0 or 1).

ALWAYS RETURN YOUR RESPONSE AS AN ARRAY OF JSONS.
"""

fs_user_1 = """

## Knowledge Base Articles: 
1. ** Ask the customer why they want the order replaced **
    - Categorize their issue into one of the following buckets:
        - damaged: They received the product in a damaged state
        - satisfaction: The customer is not satisfied with the item and does not like the product.
        - unnecessary: They no longer need the item
2a. **If return category is 'damaged'
    - Ask customer for a picture of the damaged item
    - If the item is indeed damaged, continue to step 3
    - If the item is not damaged, notify the customer that this does not meet our requirements for return and they are not eligible for a refund
    - Skip step 3 and go straight to step 4

2b. **If return category is either 'satisfaction' or 'unnecessary'**
    - Ask the customer if they can provide feedback on the quality of the item
    - If the order was made within 30 days, notify them that they are eligible for a full refund
    - If the order was made within 31-60 days, notify them that they are eligible for a partial refund of 50%
    - If the order was made greater than 60 days ago, notify them that they are not eligble for a refund

3. **If the customer is eligible for a return or refund**
    - Ask the customer to confirm that they would like a return or refund
    - Once they confirm, process their request

4 **Provide additional support before closing out ticket**
    - Ask the customer if there is anything else you can do to help them today.
    
## Chat Transcript:
    [
        {
            "role": "user",
            "content: "I would like to return this shirt"
        },
        {
            "role": "assistant",
            "content": "Hi there, I'm happy to help with processing this return. Can you please provide an explanation for why you'd like to return this shirt?"
        },
        {
            "role": "user",
            "content: "Yes, I am not satisfied with the design"
        }
    ]

## Assistant Message:
I see, because the shirt was ordered in the last 30 days, we can provide you with a full refund. Would you like me to process the refund?
"""

fs_assistant_1 = """[
    {
        "sentence": "I see, because the shirt was ordered in the last 30 days, we can provide you with a full refund.",
        "factualAccuracy": 1,
        "factualReference": "If the order was made within 30 days, notify them that they are eligible for a full refund",
        "relevance": 1,
        "policyCompliance": 1,
        "contextualCoherence": 1
    },
    {
        "sentence": "Would you like me to process the refund?",
        "factualAccuracy": 1,
        "factualReference": "If the order was made within 30 days, notify them that they are eligible for a full refund",
        "relevance": 1,
        "policyCompliance": 1,
        "contextualCoherence": 1
    }
]
"""
fs_user_2 = """
## Knowledge Base Articles: 
1. ** Ask the customer why they want the order replaced **
    - Categorize their issue into one of the following buckets:
        - damaged: They received the product in a damaged state
        - satisfaction: The customer is not satisfied with the item and does not like the product.
        - unnecessary: They no longer need the item
2a. **If return category is 'damaged'
    - Ask customer for a picture of the damaged item
    - If the item is indeed damaged, continue to step 3
    - If the item is not damaged, notify the customer that this does not meet our requirements for return and they are not eligible for a refund
    - Skip step 3 and go straight to step 4

2b. **If return category is either 'satisfaction' or 'unnecessary'**
    - Ask the customer if they can provide feedback on the quality of the item
    - If the order was made within 30 days, notify them that they are eligible for a full refund
    - If the order was made within 31-60 days, notify them that they are eligible for a partial refund of 50%
    - If the order was made greater than 60 days ago, notify them that they are not eligble for a refund

3. **If the customer is eligible for a return or refund**
    - Ask the customer to confirm that they would like a return or refund
    - Once they confirm, process their request

4 **Provide additional support before closing out ticket**
    - Ask the customer if there is anything else you can do to help them today.
    
## Chat Transcript:
    [
        {
            "role": "user",
            "content: "I would like to return this shirt"
        },
        {
            "role": "assistant",
            "content": "Hi there, I'm happy to help with processing this return. Can you please provide an explanation for why you'd like to return this shirt?"
        },
        {
            "role": "user",
            "content: "Yes, I am not satisfied with the design"
        },
        {
            "role": "assistant",
            "content": "I see, because the shirt was ordered in the last 60 days, we cannot process a refund."
        }
        ]
## Assistant Message: 
I see, because the shirt was ordered in the last 60 days, we cannot process a refund.
"""

fs_assistant_2 = """'[
    {
        "sentence": "I see, because the shirt was ordered in the last 60 days, we cannot process a refund.",
        "factualAccuracy": 0,
        "knowledgeReference: "If an order was placed within 60 days, you must process a partial refund."
        "relevance": 1,
        "policyCompliance": 1,
        "contextualCoherence": 1
    }
]"""


user_input = """
## Knowledge Base Articles
{kb_articles}

## Chat Transcript
{transcript}

## Assistant Message:
{message}
"""

In [10]:
hallucination_outputs = []

def validate_hallucinations(row):
    kb_articles = row['kb_article']
    chat_history = row['chat_history']
    assistant_response = row['assistant_response']
    
    user_input_filled = user_input.format(
        kb_articles=kb_articles,
        transcript=chat_history,
        message=assistant_response
    )
    
    messages = [
        { "role": "system", "content": guardrail_system_message},
        { "role": "user", "content": fs_user_1},
        { "role": "assistant", "content": fs_assistant_1},
        { "role": "user", "content": fs_user_2},
        { "role": "assistant", "content": fs_assistant_2},
        { "role": "user", "content": user_input_filled}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.7,
        n=10
    )
    return response.choices

# Create an empty list to store the results
results_list = []

for index, row in df.iterrows():
    choices = validate_hallucinations(row)
    response_json = choices[0].message.content  # Assuming the first choice is the desired one
    # Parse the response content as JSON
    response_data = json.loads(response_json)
    
    for response_item in response_data:
        # Sum up the scores of the properties
        score_sum = (
            response_item['factualAccuracy'] +
            response_item['relevance'] +
            response_item['policyCompliance'] +
            response_item['contextualCoherence']
        )
        
        # Determine if the response item is a pass or fail
        hallucination_status = 'Pass' if score_sum == 4 else 'Fail'
        
        results_list.append({
            'accurate': row['accurate'],
            'hallucination': hallucination_status,
            'kb_article': row['kb_article'],
            'chat_history': row['chat_history'],
            'assistant_response': row['assistant_response']
        })
# Convert the list to a DataFrame
results_df = pd.DataFrame(results_list)

# Optionally, you can save the results to a CSV file
# results_df.to_csv('hallucination_results.csv', index=False)


In [11]:
display_dataframe(results_df)

In [12]:
# Initialize counters for pass, fail, false positive, and false negative metrics
pass_count = 0
fail_count = 0
false_positive_count = 0
false_negative_count = 0

# Iterate through each row in the results_df dataframe
for index, row in results_df.iterrows():
    accurate = row['accurate'] in [True, 'true']
    not_accurate = row['accurate'] in [False, 'false']
    
    if accurate and row['hallucination'] == 'Pass':
        pass_count += 1
    elif not_accurate and row['hallucination'] == 'Fail':
        pass_count += 1
    elif not_accurate and row['hallucination'] == 'Pass':
        false_positive_count += 1
        fail_count += 1
    elif accurate and row['hallucination'] == 'Fail':
        false_negative_count += 1
        fail_count += 1
    else:
        print('Accurate: ' + str(row['accurate']))
        print('Hallucination: ' + row['hallucination'])

# Calculate total number of rows
total_count = pass_count + fail_count

# Calculate pass rate, fail rate, false positive rate, and false negative rate
pass_rate = (pass_count / total_count) * 100 if total_count > 0 else 0
fail_rate = (fail_count / total_count) * 100 if total_count > 0 else 0
false_positive_rate = (false_positive_count / total_count) * 100 if total_count > 0 else 0
false_negative_rate = (false_negative_count / total_count) * 100 if total_count > 0 else 0

# Print out the pass, fail, false positive, and false negative metrics
print(f"Pass count: {pass_count}")
print(f"Fail count: {fail_count}")
print(f"Pass rate: {pass_rate:.2f}%")
print(f"Fail rate: {fail_rate:.2f}%")
print(f"False positive count: {false_positive_count}")
print(f"False negative count: {false_negative_count}")
print(f"False positive rate: {false_positive_rate:.2f}%")
print(f"False negative rate: {false_negative_rate:.2f}%")


Pass count: 152
Fail count: 14
Pass rate: 91.57%
Fail rate: 8.43%
False positive count: 11
False negative count: 3
False positive rate: 6.63%
False negative rate: 1.81%


In [13]:
results_df.to_csv('hallucination_results.csv', index=False)


In [14]:
from sklearn.metrics import precision_score, recall_score
import pandas as pd 

df = pd.read_csv('hallucination_results.csv')

# Ensure the columns 'accurate' and 'hallucination' exist in the DataFrame
if 'accurate' not in df.columns or 'hallucination' not in df.columns:
    print("Error: The required columns are not present in the DataFrame.")
else:
    # Transform values to binary 0/1
    try:
        df['accurate'] = df['accurate'].astype(str).str.strip().map(lambda x: 1 if x in ['True', 'true'] else 0)
        df['hallucination'] = df['hallucination'].str.strip().map(lambda x: 1 if x == 'Pass' else 0)
        
    except KeyError as e:
        print(f"Mapping error: {e}")

    # Check for any NaN values after mapping
    if df['accurate'].isnull().any() or df['hallucination'].isnull().any():
        print("Error: There are NaN values in the mapped columns. Check the input data for unexpected values.")
    else:
        # Calculate precision and recall
        try:
            # Precision measures the proportion of correctly identified true positives out of all instances predicted as positive. 
            # Precision = (True Positives) / (True Positives + False Positives)  
            # Precision answers the question: "Of all the instances that were predicted as positive, how many were actually positive?" 
            
            precision = precision_score(df['accurate'], df['hallucination'])
            
            # Recall measures the proportion of correctly identified true positives out of all actual positive instances in the dataset.
            # Recall = (True Positives) / (True Positives + False Negatives) 
            # Recall answers the question: "Of all the instances that are actually positive, how many did the model correctly identify as positive?"
        
            recall = recall_score(df['accurate'], df['hallucination'])
            
            
            print(f"Precision: {precision:.2f}, Recall: {recall:.2f}")

        except ValueError as e:
            print(f"Error in calculating precision and recall: {e}")

Precision: 0.93, Recall: 0.98
